In [9]:
import numpy as np
import json,glob
from scipy.interpolate import interp2d
from scipy.ndimage.filters import gaussian_filter
from plot_corner import getBounds

# macros_bayes.txt

In [2]:
# Model neglecting both correlations and negative spins
# Loop across parallel runs and record the final dynesty evidences for each run
logz_noEvol_noNeg = np.array([])
fs = glob.glob("../code/dynesty_output/dynesty_results_noEvol_noNeg_job?.npy")
for f in fs:
    logz_noEvol_noNeg = np.append(logz_noEvol_noNeg,np.load(f,allow_pickle=True)[()]['logz'][-1])

# Model neglecting correlations
logz_noEvol = np.array([])
fs = glob.glob("../code/dynesty_output/dynesty_results_noEvol_job?.npy")
for f in fs:
    logz_noEvol = np.append(logz_noEvol,np.load(f,allow_pickle=True)[()]['logz'][-1])

# Model neglecting negative spins
logz_evol_noNeg = np.array([])
fs = glob.glob("../code/dynesty_output/dynesty_results_noNeg_job?.npy")
for f in fs:
    logz_evol_noNeg = np.append(logz_evol_noNeg,np.load(f,allow_pickle=True)[()]['logz'][-1])

# Model including negative spins and spin-q correlations
logz_evol = np.array([])
fs = glob.glob("../code/dynesty_output/dynesty_results_job?.npy")
for f in fs:
    logz_evol = np.append(logz_evol,np.load(f,allow_pickle=True)[()]['logz'][-1])

In [3]:
# Helper function to compute both the mean and std deviation of the log-Bayes between two hypotheses
def getBayes(logzs_A,logzs_B):
    bayes = logzs_A-np.mean(logzs_B)
    return np.mean(bayes), np.std(bayes)

with open('macros_bayes.txt','w') as macrofile:
    
    mean,std = getBayes(logz_noEvol_noNeg,logz_noEvol_noNeg)
    macrofile.write("\\newcommand{{\\evidenceNoEvolNoNeg}}{{{0:.2f}}}".format(mean))
    macrofile.write("\n")
    macrofile.write("\\newcommand{{\\evidenceNoEvolNoNegError}}{{{0:.2f}}}".format(std))
    macrofile.write("\n")
    
    mean,std = getBayes(logz_noEvol,logz_noEvol_noNeg)
    macrofile.write("\\newcommand{{\\evidenceNoEvolYesNeg}}{{{0:.2f}}}".format(mean))
    macrofile.write("\n")
    macrofile.write("\\newcommand{{\\evidenceNoEvolYesNegError}}{{{0:.2f}}}".format(std))
    macrofile.write("\n")
    
    mean,std = getBayes(logz_evol_noNeg,logz_noEvol_noNeg)
    macrofile.write("\\newcommand{{\\evidenceYesEvolNoNeg}}{{{0:.2f}}}".format(mean))
    macrofile.write("\n")
    macrofile.write("\\newcommand{{\\evidenceYesEvolNoNegError}}{{{0:.2f}}}".format(std))
    macrofile.write("\n")
    
    mean,std = getBayes(logz_evol,logz_noEvol_noNeg)
    macrofile.write("\\newcommand{{\\evidenceYesEvolYesNeg}}{{{0:.2f}}}".format(mean))
    macrofile.write("\n")
    macrofile.write("\\newcommand{{\\evidenceYesEvolYesNegError}}{{{0:.2f}}}".format(std))
    macrofile.write("\n")
    
    mean,std = getBayes(logz_evol,logz_evol_noNeg)
    macrofile.write("\\newcommand{{\\bayesNegWhenCorrelated}}{{{0:.2f}}}".format(mean))
    macrofile.write("\n")

# macros_pe_evol.txt

In [5]:
# Record median and 90% credible uncertainties for parameters of the spin vs. q model
samps = np.load('./../code/output/processed_emcee_samples_plPeak_r00.npy')
with open('macros_pe_evol.txt','w') as macrofile:
    
    macrofile.write("\\newcommand{{\\evolMean}}{{{0:.2f}^{{+{1:.2f}}}_{{-{2:.2f}}}}}".format(*getBounds(samps[:,7])))
    macrofile.write("\n")
    macrofile.write("\\newcommand{{\\evolSigma}}{{{0:.2f}^{{+{1:.2f}}}_{{-{2:.2f}}}}}".format(*getBounds(10.**samps[:,8])))                
    macrofile.write("\n")
    macrofile.write("\\newcommand{{\\evolAlpha}}{{{0:.2f}^{{+{1:.2f}}}_{{-{2:.2f}}}}}".format(*getBounds(samps[:,9])))                
    macrofile.write("\n")
    macrofile.write("\\newcommand{{\\evolBeta}}{{{0:.2f}^{{+{1:.2f}}}_{{-{2:.2f}}}}}".format(*getBounds(samps[:,10])))                
    macrofile.write("\n")
    macrofile.write("\\newcommand{{\\evolBetaq}}{{{0:.1f}^{{+{1:.1f}}}_{{-{2:.1f}}}}}".format(*getBounds(samps[:,5])))                
    macrofile.write("\n")
    
    # Percentage of samples with negative alpha
    alphas = samps[:,9]
    percentNegative = (100.*np.where(alphas<0)[0].size/alphas.size)
    macrofile.write("\\newcommand{{\\percentAlphaNegative}}{{{0:.1f}\\%}}".format(percentNegative)) 

# macros_pe_evol_altEvents.txt

In [7]:
# Record information about alpha and beta parameters when re-running with different sets of events

samps_no190412 = np.load('./../code/output/processed_emcee_samples_plPeak_no190412_r00.npy')
samps_no190517 = np.load('./../code/output/processed_emcee_samples_plPeak_no190517_r00.npy')
samps_no190412_no190517 = np.load('./../code/output/processed_emcee_samples_plPeak_no190412_no190517_r00.npy')
samps_w190814 = np.load('./../code/output/processed_emcee_samples_plPeak_w190814_r00.npy')

with open('macros_pe_evol_altEvents.txt','w') as macrofile:
    
    # Neglecting GW190517
    alphas = samps_no190517[:,9]
    percent_negative = 100.*alphas[alphas<0].size/alphas.size
    macrofile.write("\\newcommand{{\\percentAlphaNegativeNoOhFiveSeventeen}}{{{0:.1f}\\%}}".format(percent_negative)) 
    macrofile.write('\n')
    
    # Neglecting GW190412
    alphas = samps_no190412[:,9]
    percent_negative = 100.*alphas[alphas<0].size/alphas.size
    macrofile.write("\\newcommand{{\\percentAlphaNegativeNoOhFourTwelve}}{{{0:.1f}\\%}}".format(percent_negative)) 
    macrofile.write('\n')
    
    # Neglecting GW190517 and GW190412
    alphas = samps_no190412_no190517[:,9]
    percent_negative = 100.*alphas[alphas<0].size/alphas.size
    macrofile.write("\\newcommand{{\\percentAlphaNegativeNoOhFourTwelveOrOhFiveSeventeen}}{{{0:.1f}\\%}}".format(percent_negative)) 
    macrofile.write('\n')
    
    # Including GW190814
    alphas = samps_w190814[:,9]
    betas = samps_w190814[:,10]
    percent_negative = 100.*alphas[alphas<0].size/alphas.size
    percent_negative_betas = 100.*betas[betas<0].size/betas.size
    macrofile.write("\\newcommand{{\\percentAlphaNegativeWithOhEightFourteen}}{{{0:.1f}\\%}}".format(percent_negative)) 
    macrofile.write('\n')
    macrofile.write("\\newcommand{{\\percentBetaNegativeWithOhEightFourteen}}{{{0:.1f}\\%}}".format(percent_negative_betas))

# macros_pe_noEvol.txt

In [8]:
# Parameter estimates when excluding the possibility of spin-q correlations
samps_no_evol = np.load('../code/output/processed_emcee_samples_plPeak_noEvol_r00.npy')
with open('macros_pe_noEvol.txt','w') as macrofile:
    macrofile.write("\\newcommand{{\\noEvolMean}}{{{0:.2f}^{{+{1:.2f}}}_{{-{2:.2f}}}}}".format(*getBounds(samps_no_evol[:,7])))
    macrofile.write("\n")
    macrofile.write("\\newcommand{{\\noEvolSigma}}{{{0:.2f}^{{+{1:.2f}}}_{{-{2:.2f}}}}}".format(*getBounds(10.**samps_no_evol[:,8])))                
    macrofile.write("\n")
    macrofile.write("\\newcommand{{\\noEvolBetaq}}{{{0:.1f}^{{+{1:.1f}}}_{{-{2:.1f}}}}}".format(*getBounds(samps_no_evol[:,5])))  

# macros_pe_inj.txt

In [10]:
# Information about results of mock injection study
injection_samps_plPeak = np.load('../injection-study/processed_emcee_samples_injection_plPeak_r02.npy')
alphas_inj = injection_samps_plPeak[:,-2]
betas_inj = injection_samps_plPeak[:,-1]

# We want to compute the probability contour along which lies the point (alpha,beta) = (0,0)
# First, take a 2D histogram of our samples and smooth slightly with a gaussian kernel
xgrid = np.linspace(-1.5,1,100)
ygrid = np.linspace(-2,1.5,99)
dx = xgrid[1]-xgrid[0]
dy = ygrid[1]-ygrid[0]
heights,edgex,edgey = np.histogram2d(alphas_inj,betas_inj,bins=(xgrid,ygrid))
heights = gaussian_filter(heights,2.5)
heights /= np.sum(heights)*dx*dy

# Use resulting heights to build an interpolant for the 2D probability distribution p(alpha,beta)
# and evaluate at origin
interpolator = interp2d((xgrid[1:] + xgrid[:-1])/2,(ygrid[1:] + ygrid[:-1])/2,heights.T)
height_at_origin = interpolator(0,0)[0]

# Next, calculate cumulative probability as a function of probability density,
# integrating outwards from the max-probability point in the (alpha,beta) plane
heights_large_to_small = np.sort(heights.reshape(-1))[::-1]
cdf = np.cumsum(heights_large_to_small)*dx*dy

# Get integrated probability inside the contour intersecting (alpha,beta) = (0,0)
percentile_at_origin = 100.*np.interp(height_at_origin,heights_large_to_small[::-1],cdf[::-1])

# Write macro file
with open('macros_pe_inj.txt','w') as macrofile:
    
    macrofile.write("\\newcommand{{\\injMean}}{{{0:.2f}^{{+{1:.2f}}}_{{-{2:.2f}}}}}".format(*getBounds(injection_samps_plPeak[:,7])))
    macrofile.write("\n")
    macrofile.write("\\newcommand{{\\injSigma}}{{{0:.2f}^{{+{1:.2f}}}_{{-{2:.2f}}}}}".format(*getBounds(10.**injection_samps_plPeak[:,8])))                
    macrofile.write("\n")
    macrofile.write("\\newcommand{{\\injQuantileAtOrigin}}{{{0:.0f}}}".format(percentile_at_origin))                
    macrofile.write("\n")

# macros_ppc.txt

In [11]:
reweightedDict = np.load('reweighted_samples_noEvolution.npy',allow_pickle=True)[()]
mock_q_noEvol = reweightedDict['mock_q']
mock_x_noEvol = reweightedDict['mock_chi']
resampled_q_noEvol = reweightedDict['resampled_q']
resampled_x_noEvol = reweightedDict['resampled_chi']

# Next, repeatedly draw catalogs of mock events and reweighted posteriors, computing the least-squares slope for each
# Instantiate arrays to hold results
n_catalogs = resampled_x_noEvol.shape[1]
obs_slope = np.zeros(n_catalogs)
mock_slope = np.zeros(n_catalogs)

# Loop over catalog instantiations
for i in range(n_catalogs):
    
    # Read out spins and mass ratios
    obs_qs = resampled_q_noEvol[:,i]
    mock_qs = mock_q_noEvol[:,i]
    obs_xs = resampled_x_noEvol[:,i]
    mock_xs = mock_x_noEvol[:,i]
    
    # Compute slopes and save
    X = np.transpose([np.ones(obs_qs.size),obs_qs])
    b,m = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(obs_xs)
    obs_slope[i] = m
    
    X = np.transpose([np.ones(mock_qs.size),mock_qs])
    b,m = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(mock_xs)
    mock_slope[i] = m

In [14]:
reweightedDict_evol = np.load('reweighted_samples_yesEvolution.npy',allow_pickle=True)[()]
resampled_q_evol = reweightedDict_evol['resampled_q']
resampled_x_evol = reweightedDict_evol['resampled_chi']
mock_q_evol = reweightedDict_evol['mock_q']
mock_x_evol = reweightedDict_evol['mock_chi']

n_catalogs = resampled_q_evol.shape[1]
obs_slope_evol = np.zeros(n_catalogs)
mock_slope_evol = np.zeros(n_catalogs)

for i in range(n_catalogs):
    
    obs_qs = resampled_q_evol[:,i]
    mock_qs = mock_q_evol[:,i]
    obs_xs = resampled_x_evol[:,i]
    mock_xs = mock_x_evol[:,i]
    
    X = np.transpose([np.ones(obs_qs.size),obs_qs])
    b,m = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(obs_xs)
    obs_slope_evol[i] = m
    
    X = np.transpose([np.ones(mock_qs.size),mock_qs])
    b,m = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(mock_xs)
    mock_slope_evol[i] = m

In [16]:
with open('macros_ppc.txt','w') as macrofile:
    
    percentage = int(100.*(np.where(obs_slope<mock_slope)[0].size/n_catalogs))
    macrofile.write('\\newcommand{{\\FracBelow}}{{{0}\\%}}'.format(percentage))
    macrofile.write('\n')
    
    mean = np.mean(obs_slope)
    macrofile.write('\\newcommand{{\\meanObsSlopeBase}}{{{0:.1f}}}'.format(mean))
    macrofile.write('\n')
    
    mean_evol = np.mean(obs_slope_evol)
    mean_evol_predicted = np.mean(mock_slope_evol)
    macrofile.write('\\newcommand{{\\meanObsSlopeEvol}}{{{0:.2f}}}'.format(mean_evol))
    macrofile.write('\n')
    macrofile.write('\\newcommand{{\\meanMockSlopeEvol}}{{{0:.2f}}}'.format(mean_evol_predicted))